In [23]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessageGraph
from langchain_core.runnables import RunnablePassthrough

from langchain_core.messages import HumanMessage, AIMessage


model = ChatOpenAI(temperature=0)

def is_in_list(messages):
    list = ["what is the capital of France"]

    for message in messages:
        if message.content in list:
            print("returning cached_question")
            return "passthrough2"

    print("returning ask_question")
    return "ask_question"

def should_use_cached_question(messages):
    print("should_use_cached_question")
    return "cached_question"

def cached_question(messages):
    return AIMessage("Paris Paris Paris Paris")

graph = MessageGraph()

graph.add_node("passthrough", RunnablePassthrough())
graph.add_conditional_edges("passthrough", is_in_list)

# This is annoying... I have to use a passthrough node to connect conditional edges
graph.add_node("passthrough2", RunnablePassthrough())
graph.add_conditional_edges("passthrough2", should_use_cached_question)

graph.add_node("ask_question", model)
graph.add_edge("ask_question", END)

graph.add_node("cached_question", cached_question)
graph.add_edge("cached_question", END)

graph.set_entry_point("passthrough")

In [24]:
compiled_graph = graph.compile()

In [25]:
result = compiled_graph.invoke(HumanMessage("what is the capital of France"))
print("answer:", result[-1].content)

returning cached_question
should_use_cached_question
answer: Paris Paris Paris Paris


In [26]:
HumanMessage("what is the capital of France").content

'what is the capital of France'